In [3]:
import numpy as np 
import pandas as pd

spam_df = pd.read_csv("SPAM_SMS.csv")
sms_df = pd.read_csv("spam.csv", encoding='latin-1')

In [4]:
spam_df = spam_df.assign(type='spam')
spam_df = spam_df[['text', 'type']]
spam_df

,text,type
0,"Welcome ! your have P1222 for S!ot , \r\nWeb: ...",spam
1,"My god, at least 999P rewards waiting for you\...",spam
2,"DEAR VIP <REAL NAME>, No. 1 Online Sabong Site...",spam
3,"<REAL NAME>! Today, you can win the iphone14PR...",spam
4,"Welcome ! your have P1222 for S!ot , \r\nWeb: ...",spam
...,...,...
713,[Epicwin]: Stay entertained with EpicWin's Dai...,spam
714,<REAL NAME>Sumali na at tamasahin ang 2% na un...,spam
715,Get Set for the Payday Bonanza! Claim Your 999...,spam
716,Win Big with a 228% bonus up to 10766 PHP! Dai...,spam


In [5]:
sms_df = sms_df.loc[sms_df["v1"] == "ham"]
sms_df = sms_df[["v1", "v2"]]
columns_titles = ["v2","v1"] # swap columns 
sms_df = sms_df.reindex(columns=columns_titles)
sms_df = sms_df.rename(columns={"v2": "text", "v1": "type"})
sms_df

,text,type
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
6,Even my brother is not like to speak with me. ...,ham
...,...,...
5565,Huh y lei...,ham
5568,Will Ì_ b going to esplanade fr home?,ham
5569,"Pity, * was in mood for that. So...any other s...",ham
5570,The guy did some bitching but I acted like i'd...,ham


In [16]:
merged_df = pd.concat([spam_df, sms_df])
merged_df = merged_df.drop_duplicates()
merged_df = merged_df.sample(frac=1) #shuffle
merged_df['label_num'] = merged_df.type.map({'ham':0, 'spam':1})
merged_df = merged_df.drop(columns=['type'])
merged_df['label_num'].value_counts()

label_num
0    4516
1     660
Name: count, dtype: int64

In [20]:
x = merged_df.text
y = merged_df.label_num

(5176,) (5176,)


In [22]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=39)

(3882,)
(1294,)
(3882,)


In [26]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
x_train_dtm = vect.fit_transform(x_train)
x_test_dtm = vect.transform(x_test)

In [24]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_train_dtm, y_train)

MultinomialNB()

In [29]:
from sklearn import metrics
y_pred = nb.predict(x_test_dtm)
metrics.accuracy_score(y_test, y_pred)


0.9945904173106646